<a href="https://colab.research.google.com/github/ktynski/Marketing_Automations_Notebooks_With_GPT/blob/main/Exploring_Multi_Agent_AI_Collaboration_for_Iterative_Invention%2C_Critique%2C_and_Synthesis_Public_By_Kristin_Frac_tl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Autonomous Ideation Agent for Novel Invention Generation**

---

> ## **Project Overview**
> *Invention Ideation through AI-Driven Autonomy*

---

> ## **Technical Approach**
>
> **AI-Driven Invention Generation**
> Simulating creative genius, our agent employs cutting-edge AI to forge **unconstrained** invention ideas across domains.
>
> **Iterative Learning Framework**
> Learning from both successes and failures, the agent adapts, evolving its understanding of **innovation landscapes**.
>
> **Multidimensional Idea Evaluation**
> Quantitative metrics and qualitative insights converge to **evaluate** feasibility, innovation, complexity, cost, and market potential.
>
> **Failure Analysis and Refinement**
> Failures breed growth. Dissecting setbacks, the agent refines strategies, ensuring each iteration **surpasses the last**.

---

> ## **Significance**
>
> 1. **AI-Powered Creativity**
>    - Synthesizing unconventional ideas **beyond boundaries**.
>
> 2. **Iterative Adaptation**
>    - Continuous improvement driven by **evolving preferences**.
>
> 3. **Qualitative-Quantitative Fusion**
>    - Holistic evaluation enlightening **potential and feasibility**.
>
> 4. **Failure-Driven Innovation**
>    - Setbacks as **stepping stones** for enhanced ideation.

---

> ## **Implications**
>
> Reshaping innovation in an evolving technological era, the project fuses AI, creativity, innovation, and the [`simpleaichat` library](https://github.com/minimaxir/simpleaichat) by [@minimaxir](https://github.com/minimaxir). Applications span diverse fields, envisioning the **future of ideation**.

---

> ## **Technology Leveraged**
>
> An update to ChatGPT on June 13th, 2023 allows the user to set a predefined schema to have ChatGPT output data according to that schema and/or take in an input schema and respond better to that data. This "function calling," as OpenAI calls it, can be used as a form of tools, but the schema, enabled by a JSON-finetuning of ChatGPT, is much more useful for typical generative AI use cases, particularly when not using GPT-4.
>
> OpenAI's [official demos](https://github.com/openai/openai-cookbook/blob/main/examples/How_to_call_functions_with_chat_models.ipynb) for this feature are complicated, but with `simpleaichat`, it's very easy to support placing your own data.
>
> **NOTE: Ensuring input and output follows a complex predefined structure is very new in the field of prompt engineering, and although it is very powerful, your mileage may vary.**

---


## Install and Import Required Libraries

In [ ]:
!pip install -q simpleaichat
!pip install openai

from simpleaichat import AIChat
import orjson
from rich.console import Console
from getpass import getpass

from typing import List, Literal, Optional, Union
from pydantic import BaseModel, Field

In [ ]:
from simpleaichat import AIChat
import os
import pandas as pd
import openai
import time
from IPython.display import display, Markdown
from IPython.core.display import HTML
from PIL import Image as PILImage
from io import BytesIO
import requests

# Replace YOUR_API_KEY with the actual OpenAI API key
os.environ['OPENAI_API_KEY'] = "Your OpenAI API Key"

# Function to generate image from text
def generate_image_from_text(prompt_text, api_key, imagefilename):
    r = requests.post('https://clipdrop-api.co/text-to-image/v1',
        files = {'prompt': (None, prompt_text, 'text/plain')},
        headers = {'x-api-key': api_key}
    )

    if r.ok:
        image_data = r.content
        image = PILImage.open(BytesIO(image_data))

        # Replace spaces with underscores and remove special characters for filename
        print(f"Image Filename: {imagefilename}")
        filename = imagefilename.replace(" ", "_") + ".png"

        # Save the image
        image.save(filename)
        display(Markdown(f"### Image Prototype Rendering Using Stability.AI"))
        display(image)
        return filename
    else:
        r.raise_for_status()



# Define separate AIChat instances for different tools
params_audience_analyzer = {"model": "gpt-3.5-turbo-16k", "temperature": 0.5, "max_tokens": 100}
params_ideator= {"model": "gpt-4", "temperature": 0.2, "max_tokens": 100}
params_critiquer = {"model": "gpt-3.5-turbo-16k", "temperature": 0.7, "max_tokens": 100}
params_integrator = {"model": "gpt-4", "temperature": 0.7, "max_tokens": 600}
params_scorer = {"model": "gpt-3.5-turbo-16k", "temperature": 0.2, "max_tokens": 10, "logit_bias": {str(k): 10 for k in range(1, 9)}}
params_learning_agent = {"model": "gpt-3.5-turbo-16k", "temperature": 0.5, "max_tokens": 200}
params_image_prompter = {"model": "gpt-4", "temperature": 0.5, "max_tokens": 200}
params_markdown_formatter = {"model": "gpt-4", "temperature": 0.5, "max_tokens": 800}



# Define a new AIChat instance for analyzing the target audience
ai_audience_analyzer = AIChat(system="You are an expert in analyzing the target audience for various products and services. Based on a given query, provide insights into the potential target audience, their preferences, needs, and any other relevant details that would be helpful for an inventor who is trying to create a blockbuster invention for this audience.", params=params_audience_analyzer, console=False)
ai_ideator = AIChat(system="You are an invention ideation genius. Your goal is to come up with a single genius blockbuster innovative idea for the given invention category. Start your response with Invention Idea:",params=params_ideator, console=False)
ai_critiquer = AIChat(system="You are a critiquer.You poke holes in an idea and think of counterfactuals. You also make suggestions for solving found issues and signficicant improvements. Start your response with Critique:",params=params_critiquer, console=False)
ai_integrator = AIChat(system="You are an integrator of invention and invention critique. You provide an improved version of the idea based on the original idea and the critique You return an updated idea based on your synthesis of the critique. Start your response with Revised Idea:",params=params_integrator, console=False)
ai_scorer = AIChat(system="You are a scorer providing only a single 1-9 score for the quality of the invention idea. You provide no other text before or after your score.",params=params_scorer, console=False)
ai_scorer_originality = AIChat(system="You are a scorer for the invention's originality, providing only a single 1-9 score.You provide no other text before or after your score. ", params=params_scorer, console=False)
ai_scorer_practicality = AIChat(system="You are a scorer for the invention's practicality, providing only a single 1-9 score. You provide no other text before or after your score.", params=params_scorer, console=False)
ai_scorer_impact = AIChat(system="You are a scorer for the invention's impact, providing only a single 1-9 score. You provide no other text before or after your score.", params=params_scorer, console=False)
ai_scorer_alignment = AIChat(system="You are a scorer for the invention's alignment with target audience, providing only a single 1-9 score.You provide no other text before or after your score. ", params=params_scorer, console=False)
ai_scorer_aesthetic = AIChat(system="You are a scorer for the invention's aesthetic and design quality, providing only a single 1-9 score.You provide no other text before or after your score. ", params=params_scorer, console=False)
ai_learning_agent = AIChat(system="You are a learning agent responsible for analyzing previous iterations, identifying patterns, and providing insights to guide the invention process. Your readout should be directive and non-generic, preferencing specific recommendations for the product itself, and not the process of improvement", params=params_learning_agent, console=False)
ai_image_prompter = AIChat(system="You are an expert in crafting prompts for text-to-image models like Midjourney/Dalle2. Based on the given synthesized idea, create a prompt that follows best practices for generating successful text-to-image outputs.", params=params_image_prompter, console=False)
ai_markdown_formatter = AIChat(system="You are an expert in formatting text into beautiful markdown. Take the given idea and present it in a well-organized and visually appealing markdown format, adding any missing details as needed.", params=params_markdown_formatter, console=False)


# Number of critique and synthesis iterations per idea
NUM_STAGES = 3

# Define thresholds for various scores
QUALITY_THRESHOLD = 8.0
ORIGINALITY_THRESHOLD = 8.0
PRACTICALITY_THRESHOLD = 9.0
IMPACT_THRESHOLD = 9.0
ALIGNMENT_THRESHOLD = 7.0
AESTHETIC_THRESHOLD = 7.0

# Define DataFrame columns
columns = ['Iteration', 'Query', 'Target Audience', 'Idea']
for i in range(1, NUM_STAGES + 1):
    columns += [f'Critique{i}', f'Synthesis{i}']
columns += ['Quality Score', 'Originality', 'Practicality', 'Impact', 'Alignment', 'Aesthetic', 'Learning_Agent_Insights']

# Initialize DataFrame with columns
progress_df = pd.DataFrame(columns=columns)

iteration_count = 0

# Outer loop to continue until quality threshold is met
while True:
    time.sleep(60)
    iteration_count += 1
    print(f"Iteration {iteration_count}")
    query = "Invent a blockbuster toy for Autistic Teens"

    # Analyze the target audience based on the query
    audience_details = ai_audience_analyzer(f"Analyze the target audience for the query: {query}")

    # Collect and analyze previous iterations (if available)
    if iteration_count > 1:
        previous_ideas = progress_df[['Idea'] + [f'Critique{i}' for i in range(1, NUM_STAGES + 1)]]
        idea_insights = ai_learning_agent(f"Analyze previous iterations and provide insights and learnings that would make the next round of invention ideas more fruitful and likely to succeed in a blockbuster invention. {previous_ideas.to_string()}")
        previous_critiques = progress_df[[f'Critique{i}' for i in range(1, NUM_STAGES + 1)]]
        critique_insights = ai_learning_agent(f"Analyze previous critiques and provide insights: {previous_critiques.to_string()}")

    else:
        idea_insights = "No previous iterations to analyze."
        critique_insights = "No previous critiques to analyze."

    # Generate creative ideas, incorporating insights from the learning agent
    idea = ai_ideator(f"""Generate creative ideas for {query} based on a deep understanding of the needs, wants, and specific challenges of the target audience: {audience_details}.
    Make good use of the Insights from previous ideas so you do not repeat your mistakes or suggest an identical failed idea and can iteratively improve. In some cases there will be opportunities to think laterally and combine elements of previous ideas. Previous Ideas: {idea_insights}.
    Also leverage learnings from previous idea critiques to inform and improve subsequent ideas by not repeating mistakes and having a deeper ongoing understanding of what will be required for a highly successful idea. Previous Critiques: {critique_insights}""")

    # Add a new row for this idea
    row_data = {'Iteration': iteration_count, 'Target Audience': audience_details, 'Query': query, 'Idea': idea, 'Learning Agent Idea Insights': idea_insights,  'Learning Agent Critique Insights': critique_insights}
    synthesis = idea

    # Loop through critique and synthesis stages
    for stage in range(1, NUM_STAGES + 1):
        time.sleep(60)
        critique = ai_critiquer(f"Critique the given idea: {synthesis}")
        synthesis = ai_integrator(f"""Synthesize the given idea and critique into an improved version that considers the suggestions and feedback.
        The output should be a revised idea starting with Revised Idea: Data to use for revised idea:  Previous iteration idea:{synthesis} \n Previous iteration critique:{critique}""")
        row_data[f'Critique{stage}'] = critique
        row_data[f'Synthesis{stage}'] = synthesis

    # Evaluate the final synthesis
    row_data['Quality Score'] = float(ai_scorer(f"Evaluate the quality of this synthesis: {synthesis}"))
    row_data['Originality'] = float(ai_scorer_originality(f"Score the originality of this synthesis: {synthesis}"))
    row_data['Practicality'] = float(ai_scorer_practicality(f"Score the practicality of this synthesis: {synthesis}"))
    row_data['Impact'] = float(ai_scorer_impact(f"Score the impact of this synthesis: {synthesis}"))
    row_data['Alignment'] = float(ai_scorer_alignment(f"Score the alignment of this synthesis: {synthesis} based on value to the target audience {audience_details}"))
    row_data['Aesthetic'] = float(ai_scorer_aesthetic(f"Score the aesthetic of this synthesis: {synthesis}"))

    # Append the row data to the DataFrame
    progress_df = pd.concat([progress_df, pd.DataFrame([row_data])], ignore_index=True)

    # Check whether all thresholds have been met
    if (row_data['Quality Score']  >= QUALITY_THRESHOLD and
        row_data['Originality'] >= ORIGINALITY_THRESHOLD and
        row_data['Practicality'] >= PRACTICALITY_THRESHOLD and
        row_data['Impact'] >= IMPACT_THRESHOLD and
        row_data['Alignment'] >= ALIGNMENT_THRESHOLD and
        row_data['Aesthetic'] >= AESTHETIC_THRESHOLD):
        final_synthesis = synthesis

        # Generate the image prompt
        image_prompt = ai_image_prompter(f"Create a text-to-image prompt based on the final synthesized idea. Make sure to describe the idea in terms that would be needed for an artist to render it visually, showing off it's specific features: {final_synthesis}")
        markdown_text = ai_markdown_formatter(f"Please further flesh out the finalized idea if any details or specifics are missing. Return your answer in beautiful Markdown making full use of all relevant markdown formats. Idea for fleshing out and markdown enhancement: {final_synthesis}")

        # Generate the image
        api_key_image = "Your ClipDrop API Key"
        filename = "final_idea_image"
        image_filename = generate_image_from_text(image_prompt, api_key_image, filename)

        # Display the final idea, its scores, and the image
        display(Markdown(f"### Final Synthesis:"))
        display(Markdown(markdown_text))
        display(Markdown(f"### Evaluation Scores:"))
        table_html = f"""
        <table style="width:50%">
          <tr>
            <th>Metric</th>
            <th>Score</th>
          </tr>
          <tr>
            <td>Quality</td>
            <td>{row_data['Quality Score']}</td>
          </tr>
          <tr>
            <td>Originality</td>
            <td>{row_data['Originality']}</td>
          </tr>
          <tr>
            <td>Practicality</td>
            <td>{row_data['Practicality']}</td>
          </tr>
          <tr>
            <td>Impact</td>
            <td>{row_data['Impact']}</td>
          </tr>
          <tr>
            <td>Alignment</td>
            <td>{row_data['Alignment']}</td>
          </tr>
          <tr>
            <td>Aesthetic</td>
            <td>{row_data['Aesthetic']}</td>
          </tr>
        </table>
        """

        display(HTML(table_html))

        # Save the DataFrame
        progress_df.to_csv('Full_Ideas_Dataframe.csv')
        print("Final DataFrame saved to 'Full_Ideas_Dataframe.csv'")
        break

print(f"Final Synthesis: {synthesis}")








Iteration 1
Iteration 2
Iteration 3


In [ ]:
image_prompt   #print this and try it in Midjourney if you want an even better result. Stability AI's latest model (Via ClipDrop) is good, but not nearly as good as Midjourney.

In [ ]:
progress_df    #print the dataframe of inventions for a quick look

In [ ]:
progress_df.to_csv('inventioneval.csv')

## Previous Approach I Tried (More prone to errors, perhaps somewhat inferior to the above approach but you are welcome to try it as well)

In [ ]:
from simpleaichat import AIChat
from pydantic import BaseModel, Field
import pandas as pd
from IPython.display import display, Markdown
import time
import requests
from PIL import Image as PILImage
from io import BytesIO

from simpleaichat import AIChat
from pydantic import BaseModel, Field
import pandas as pd
from IPython.display import display, Markdown
import time
import requests
from PIL import Image as PILImage
from io import BytesIO
import openai




def generate_image_from_text(prompt_text, api_key, imagefilename):
    r = requests.post('https://clipdrop-api.co/text-to-image/v1',
        files = {'prompt': (None, prompt_text, 'text/plain')},
        headers = {'x-api-key': "Your Stability AI ClipDrop API Key"}
    )

    if r.ok:
        image_data = r.content
        image = PILImage.open(BytesIO(image_data))  # use PILImage instead of Image

        # Replace spaces with underscores and remove special characters for filename
        print(f"Image Filename: {imagefilename}")
        filename = imagefilename
        filename = filename.replace(" ", "_") + ".png"

        # Save the image
        image.save(filename)
        display(Markdown("#### Stability.AI Generated Prototype Rendering "))

        display(image)
        return filename
    else:
        r.raise_for_status()





def generate_invention_image_prompt(prompt, model="gpt-4", max_tokens=400, temperature=0.4):
    gpt_response = openai.ChatCompletion.create(
        model=model,
        messages=[
            {"role": "system", "content": """You will be given an extremely detailed new invention product overview. Your job is to use this to write an image prompt for Dalle2.
            The goal is to describe it in high visual detail so the AI model will be able to clearly understand exactly what to draw and how.
            Please provide specific direction on how to draw the prototype, a specific style to render it in based on your understanding of the product, and great detail
            so that the prompt you are writing when given to the image generating AI will perform accurately. Keep in mind best practices for image AI prompting for systems like midjourney or Dalle2 """},
            {"role": "user", "content": f"Product Details: {prompt}"}],
        max_tokens=max_tokens,
        n=1,
        stop=None,
        temperature=temperature,
    )
    response = gpt_response['choices'][0]['message']['content'].strip()
    return response


class Invention(BaseModel):
    """Container for invention details"""
    Name: str = Field(description="Name of the invention")
    Description: str = Field(description="A detailed description of the invention")
    Potential_Usage: str = Field(description="Possible use cases for the invention")
    Unique_Features: str = Field(description="Unique and innovative features of the invention")
    Materials_Required: str = Field(description="List of materials required for building the invention")


class Invent(BaseModel):
    """Container for multiple invention ideas"""
    Invention_1: Invention = Field(description="The first invention")
    Invention_2: Invention = Field(description="The second invention")
    Invention_3: Invention = Field(description="The third Invention")
    Invention_4: Invention = Field(description="The fourth Invention")


class Evaluate(BaseModel):
    """Evaluation information for an idea or invention"""

    feasibility_score: int = Field(
        ...,
        description="Score for feasibility (1-10): 10 indicates that the idea is highly feasible and easy to create, while 1 suggests it's nearly impossible or presents exceptional roadblocks."
    )
    feasibility_description: str = Field(
        ...,
        description="Provide a detailed analysis of the feasibility score, outlining challenges, opportunities, and potential roadblocks."
    )

    innovation_score: int = Field(
        ...,
        description="Score for innovation (1-10): 10 represents an extremely innovative concept, while 1 signifies something predictable or already existing."
    )
    innovation_description: str = Field(
        ...,
        description="Explain the innovation score by describing the novelty of the idea, comparing it to existing solutions, and analyzing its creative aspects."
    )

    complexity_score: int = Field(
        ...,
        description="Score for complexity (1-10): 10 denotes a highly complex and possibly unfeasible idea, whereas 1 indicates simplicity."
    )
    complexity_description: str = Field(
        ...,
        description="Provide an in-depth analysis of the complexity score, detailing components, interactions, and challenges in implementation."
    )

    cost_score: int = Field(
        ...,
        description="Score for production cost (1-10): 10 implies extremely high costs, while 1 means the idea would be cheap to produce."
    )
    cost_description: str = Field(
        ...,
        description="Explain the cost score, including estimates for production, materials, labor, and other factors influencing the overall cost."
    )

    market_potential_score: int = Field(
        ...,
        description="Score for market potential (1-10): 10 signifies a blockbuster idea with broad appeal, while 1 represents minimal or niche interest."
    )
    market_potential_description: str = Field(
        ...,
        description="Analyze the market potential score by examining target demographics, competition, market trends, and potential barriers to entry."
    )


class FailureReasonSummarizer(BaseModel):
    """Summarizes failure reasons based on detailed summary."""

    detailed_summary: str = Field(description="Detailed summary of failure reasons")

    def generate_concise_summary(self, ai_instance):
        try:
            concise_summary = ai_instance(f"Given your understanding of the failure reasons, for the next iteration of ideas, what should the AI try to keep in mind. Please keep this to just a few sentences of the most valuable advice gleaned from your evaluation. Advice:: {self.detailed_summary}")
            return concise_summary
        except:
            return "Failed to generate a concise failure reason summary."



class IterationDecision(BaseModel):
    """Decision for next iteration step"""
    action: str = Field(description="Action to take: 'new_ideas' or 'move_on'")
    reason: str = Field(description="Reason for the decision")



class MarketingPrompt(BaseModel):
    """Container for marketing prompt details"""
    Tagline: str = Field(description="Please simulate an expert product marketer and provide a Catchy tagline for the product that would make potential customers curious and that shows off the unique value prop of the product.")
    Description: str = Field(description="Short marketing description highlighting the product's unique features and potential usage")


class PrototypePrompt(BaseModel):
    """Container for prototype design prompt details"""
    Design_Description: str = Field(
        ...,
        description="Highly detailed and long form description of the prototype design, detailing the product's unique features and potential usage. It should be detailed enough for a factory to understand how to produce the product."
    )
    Features: str = Field(
        ...,
        description="List of key features that will be incorporated into the prototype design. Every single feature should be highly detailed and specifically described."
    )
    Materials: str = Field(
        ...,
        description="List of specific materials that will be used to build the prototype. Include the exact material or materials to be used and the reasoning for using those materials"
    )
    Functionality: str = Field(
        ...,
        description="Description of the expected functionality and capabilities of the prototype. This should be highly detailed and cover every single function. It should be clear exactly how the invention accomplishes those functions."
    )
    User_Interactions: str = Field(
        ...,
        description="Highly detailed Explanation of how users will interact with the prototype and its features"
    )
    Testing_Plan: str = Field(
        ...,
        description="Outline of the testing plan for evaluating the prototype's performance and usability"
    )



class EvaluateAndImprove(BaseModel):
    """Container for evaluating and improving selected invention"""
    Improved_Tagline: str = Field(..., description="Refined and significantly improve the tagline for marketing that captures the essence of the product")
    Improved_Description: str = Field(..., description="Improve, extend and enhance the marketing description highlighting key selling points for the best possible marketing description")
    Improved_Design_Description: str = Field(..., description="Make sure that the design description fully explains the product such that it could be given to an engineering firm and be made based on the description alone.")
    Improved_Features: str = Field(..., description="Expand on and improve the description of the features. Detail each part individually for a long-form, fleshed out understanding of every single feature and why they matter.")
    Improved_Materials: str = Field(..., description="Significantly improve and extend the selection of materials ensuring durability and cost-effectiveness and any other criteria that might improve the product further.")
    Improved_Functionality: str = Field(..., description="Improve and significantly extend the detailed explanation of the functionality and performance capabilities. Make sure the functionality is fully described and it is clear exactly how the product accomplishes that functionality.")
    Improved_User_Interactions: str = Field(..., description="Improve and extend the Clear instructions on how users will interact with the refined prototype, including user interfaces and controls, etc. This should be a highly detailed description of how the user interacts with the product to get the value.")
    Improved_Testing_Plan: str = Field(..., description="Improve and extend the Comprehensive testing plan, outlining methods, metrics, criteria, etc. for evaluating the prototype's effectiveness")




def evaluate_invention(ai, invention_name, max_retries):
    evaluation = ai(f"Evaluate the following invention: {invention_name}", output_schema=Evaluate)
    return evaluation


def find_unique_invention(
    model_name,
    api_key,
    invention_category,
    max_retries,
    success_thresholds,
    temperature=0.8,
    max_tokens = 2000
):
    ai = AIChat(
        console=False,
        save_messages=False,
        model=model_name,
        params={"temperature": temperature, "max_tokens": max_tokens},
        api_key=api_key
    )

    import pandas as pd

    columns = ['Name', 'Description', 'Potential_Usage', 'Unique_Features', 'Materials_Required', 'Feasibility', 'Innovation', 'Complexity', 'Cost', 'Failure_Reasons']
    failed_inventions_df = pd.DataFrame(columns=columns)

    retry_count_for_new_ideas = 0

    while True:
        successful_idea_found = False
        # Combine failure reasons into a prompt
        failed_inventions_prompt = "\n".join(
            f"Invention Name: {name}\nLearnings to apply to next iteration: {reasons}"
            for name, reasons in failed_inventions_df[['Name', 'Failure_Reasons']].values
        )

        # Define a maximum prompt length
        max_prompt_length = 4096  # Adjust this value as needed

        # Truncate the prompt if it's too long
        if len(failed_inventions_prompt) > max_prompt_length:
            excess_length = len(failed_inventions_prompt) - max_prompt_length
            truncated_failed_inventions_prompt = "..." + failed_inventions_prompt[excess_length:]
        else:
            truncated_failed_inventions_prompt = failed_inventions_prompt

        print(len(failed_inventions_prompt))
        prompt = f"""Please simulate a quirky an eccentric genius inventor that is known for your creativity, uniqueness, and ability to think laterally and counterfactually to come up with the best possible ideas the world has never yet seen.
        Your ideas are unique and have not been invented or heard of before. NEVER invent something that already exists. It must be Novel. Try to come up with an invention that is feasible, innovative, not impossibly complex, cost effective to produce, that serves a market need.
        A list of 4 innovative inventions in the category of {invention_category} that will be highly useful, interesting, and never seen before. It should be feasible, innovative, not very complex, and have a low cost to produce.
        Please think step by step so you come to the best possible invention idea.
        Avoid the following failed ideas and learn from what has not worked to inform your new ideas. Think laterally and consider mashups/combinations of failed ideas or parts of failed ideas that could be combined into completely unique and never before seen products.
        Never come up with an invention extremely similar to an already failed invention idea. Failed invention ideas:\n{failed_inventions_prompt}"""

        while True:
            try:
                invention_ideas = ai(prompt, output_schema=Invent)
                break  # If successful, break out of the loop
            except:
                time.sleep(20)
                error_message = "An error occurred while generating invention ideas. Retrying..."
                display(Markdown("---"))
                display(Markdown(f"**Error:** {error_message}"))
                display(Markdown("---"))
                continue

        if retry_count_for_new_ideas == max_retries:
            display(Markdown("Reached the maximum number of retries for generating new ideas."))
            continue  # Continue to the next iteration of the outer loop

        iteration_count = 1
        successful_idea_found = False

        # Loop through each invention idea
        for invention_key, invention_data in invention_ideas.items():
            name = invention_data['Name']
            description = invention_data['Description']
            potential_usage = invention_data['Potential_Usage']
            unique_features = invention_data['Unique_Features']
            materials_required = invention_data['Materials_Required']

            display(Markdown("---"))
            display(Markdown(f"## Evaluating invention - {name}"))
            display(Markdown(f"### Invention Description - {description}"))

            # Retry for evaluating invention
            retry_count_for_evaluation = 0
            while retry_count_for_evaluation < max_retries:
                try:
                    evaluation = evaluate_invention(ai, name, max_retries)
                    if evaluation is None:
                        display(Markdown("---"))  # Add a horizontal rule
                        display(Markdown("Proceeding to the next invention..."))
                        display(Markdown("---"))  # Add a horizontal rule
                        break
                    feasibility = evaluation["feasibility_score"]
                    innovation = evaluation["innovation_score"]
                    complexity = evaluation["complexity_score"]
                    cost = evaluation["cost_score"]
                    market_potential = evaluation["market_potential_score"]

                    feasibility_description = evaluation["feasibility_description"]
                    innovation_description = evaluation["innovation_description"]
                    complexity_description = evaluation["complexity_description"]
                    cost_description = evaluation["cost_description"]
                    market_potential_description = evaluation["market_potential_description"]
                    cost_description = evaluation["cost_description"]

                    display(Markdown(f"### Scores - Feasibility: {feasibility}, Innovation: {innovation}, Complexity: {complexity}, Cost: {cost}, Market Potential: {market_potential}"))

                    if (feasibility > success_thresholds['feasibility'] and
                        innovation > success_thresholds['innovation'] and
                        complexity < success_thresholds['complexity'] and
                        cost < success_thresholds['cost'] and
                        market_potential > success_thresholds['market_potential']):
                        decision = IterationDecision(action="move_on", reason="Satisfactory criteria met for this invention")
                        successful_idea_found = True
                        display(Markdown(f"**Decision: Moving on with invention {name}.**\nReason: {decision.reason}"))
                        break
                    else:
                        time.sleep(10)
                        try:
                            failure_reason_summary = ai(f"Summarize the reasons for failure of the following invention: {name}\n\n{feasibility_description}\n\n{innovation_description}\n\n{complexity_description}\n\n{cost_description}\n\n{market_potential_description}")
                        except:
                            failure_reason_summary = "Failed to generate a failure reason summary."

                        try:
                            concise_failure_reasons = ai(f"Summarize the key failure reasons from the detailed summary: {failure_reason_summary}")
                        except:
                            time.sleep(10)
                            concise_failure_reasons = "Failed to generate a concise failure reason summary."
                        display(Markdown(f'**Failure Reasons Summary:** {concise_failure_reasons}'))
                        display(Markdown("---"))
                        failed_inventions_df = pd.concat([failed_inventions_df, pd.DataFrame([{
                            'Name': name,
                            'Description': description,
                            'Potential_Usage': potential_usage,
                            'Unique_Features': unique_features,
                            'Materials_Required': materials_required,
                            'Feasibility': feasibility,
                            'Innovation': innovation,
                            'Complexity': complexity,
                            'Cost': cost,
                            'Market Potential Score': market_potential,
                            'Feasibility_Reason': feasibility_description,
                            'Innovation_Reason': innovation_description,
                            'Complexity_Reason': complexity_description,
                            'Cost_Reason': cost_description,
                            'Market Potential Reason': market_potential_description,
                            'Failure_Reasons': concise_failure_reasons
                            }])])

                        iteration_count += 1
                        failed_inventions_df.to_csv('Failed_Inventions_Dataframe.csv')
                        break  # Break the loop for this invention
                except Exception as e:
                    time.sleep(30)
                    error_message = f"An error occurred while evaluating invention: {name}. Retrying... (Attempt {retry_count_for_evaluation + 1}/{max_retries})"
                    display(Markdown(f"**Error:** {error_message}"))
                    retry_count_for_evaluation += 1
                    continue

            if retry_count_for_evaluation >= max_retries:
                display(Markdown("---"))
                display(Markdown("### Maximum Retries Reached for Evaluating Inventions"))
                display(Markdown("Reached the maximum number of retries for evaluating inventions. Moving to the next invention..."))
                display(Markdown("---"))
                continue
            failed_inventions_df.to_csv('Failed_Inventions_Dataframe.csv')


        if successful_idea_found:
            successful_idea_md = f"""
            ### Congratulations! A satisfactory invention idea has been found.
            - **Name:** {name}
            - **Description:** {description}
            - **Potential Usage:** {potential_usage}
            - **Unique Features:** {unique_features}
            - **Materials Required:** {materials_required}
            - **Feasibility Score:** {feasibility}
            - **Innovation Score:** {innovation}
            - **Complexity Score:** {complexity}
            - **Cost Score:** {cost}
            - **Market Potential Score:** {market_potential}
            - **Feasibility Reason:** {feasibility_description}
            - **Innovation Reason:** {innovation_description}
            - **Complexity Reason:** {complexity_description}
            - **Cost Reason:** {cost_description}
            - **Market Potential Reason:** {market_potential_description}
            """
            display(Markdown(successful_idea_md))

            successful_idea = {
                'Name': name,
                'Description': description,
                'Potential_Usage': potential_usage,
                'Unique_Features': unique_features,
                'Materials_Required': materials_required,
                'Feasibility_Score': feasibility,
                'Innovation_Score': innovation,
                'Complexity_Score': complexity,
                'Cost_Score': cost,
                'Market_Potential_Score': market_potential,
                'Feasibility_Reason': feasibility_description,
                'Innovation_Reason': innovation_description,
                'Complexity_Reason': complexity_description,
                'Cost_Reason': cost_description,
                'Market_Potential_Reason': market_potential_description
            }

            return successful_idea

        if retry_count_for_new_ideas >= max_retries:
            display(Markdown("---"))
            display(Markdown("### Maximum Retries Reached"))
            display(Markdown("Reached the maximum number of retries for this set of ideas. Generating new ideas for the next iteration..."))
            display(Markdown("---"))  # Add a horizontal rule
            retry_count_for_new_ideas = 0
            continue
        else:
            time.sleep(10)
            display(Markdown("---"))
            display(Markdown("### No Satisfactory Inventions Found"))
            display(Markdown("The current iteration did not yield any satisfactory invention ideas. Generating new ideas for the next iteration..."))
            display(Markdown("---"))  # Add a horizontal rule
            continue



def proceed_with_successful_idea(successful_idea, model_name, api_key):
    ai = AIChat(
        console=False,
        save_messages=False,
        model=model_name,
        params={"temperature": 0.6, "max_tokens": 4000},
        api_key=api_key
    )

    display(Markdown("## Proceeding with the Successful Idea"))
    display(Markdown(f"**Name:** {successful_idea['Name']}"))
    display(Markdown(f"**Description:** {successful_idea['Description']}"))
    display(Markdown(f"**Potential Usage:** {successful_idea['Potential_Usage']}"))
    display(Markdown(f"**Unique Features:** {successful_idea['Unique_Features']}"))
    display(Markdown(f"**Materials Required:** {successful_idea['Materials_Required']}"))

    marketing_prompt = f"Introducing the revolutionary {successful_idea['Name']}! Discover a groundbreaking solution that offers {successful_idea['Potential_Usage']} with its unique features: {successful_idea['Unique_Features']}"
    prototype_prompt = f"Design a prototype for {successful_idea['Name']}, highlighting its unique features and potential applications: {successful_idea['Potential_Usage']}. The prototype should have the following features: Feature 1, Feature 2. It should be constructed using Materials: Material 1, Material 2. The prototype's functionality should include: The prototype will be capable of performing... Users will interact with the prototype by... The testing plan should include..."

    while True:
        try:
            marketing_content = ai(marketing_prompt, output_schema=MarketingPrompt)
            break  # If successful, break out of the loop
        except:
            time.sleep(10)
            print(f"JSONDecodeError occurred. Retrying marketing content generation for marketing content...")
            continue

    display(Markdown("## Generated Marketing Content"))
    display(Markdown(f"**Tagline:** {marketing_content['Tagline']}"))
    display(Markdown(f"**Description:** {marketing_content['Description']}"))

    while True:
        try:
            prototype_design = ai(prototype_prompt, output_schema=PrototypePrompt)
            break  # If successful, break out of the loop
        except :
            time.sleep(10)
            print(f"JSONDecodeError occurred. Retrying prototype design generation...")
            continue

    display(Markdown("## Generated Prototype Design"))
    display(Markdown(f"**Design Description:** {prototype_design['Design_Description']}"))
    display(Markdown(f"**Features:** {prototype_design['Features']}"))
    display(Markdown(f"**Materials:** {prototype_design['Materials']}"))
    display(Markdown(f"**Functionality:** {prototype_design['Functionality']}"))
    display(Markdown(f"**User Interactions:** {prototype_design['User_Interactions']}"))
    display(Markdown(f"**Testing Plan:** {prototype_design['Testing_Plan']}"))


    # Iterative refinement loop
    for iteration in range(5): # 5 iterations as an example; customize as needed
        display(Markdown(f"### Iteration {iteration + 1}"))
        time.sleep(20)

        # Evaluate and improve marketing content
        marketing_evaluation_prompt = f"Evaluate and suggest improvements for the following marketing content: Tagline: {marketing_content['Tagline']}, Description: {marketing_content['Description']}"
        evaluation_and_improvement_marketing = ai(marketing_evaluation_prompt, output_schema=EvaluateAndImprove)
        marketing_content['Tagline'] = evaluation_and_improvement_marketing['Improved_Tagline']
        marketing_content['Description'] = evaluation_and_improvement_marketing['Improved_Description']

        display(Markdown("#### Updated Marketing Content"))
        display(Markdown(f"**Tagline:** {marketing_content['Tagline']}"))
        display(Markdown(f"**Description:** {marketing_content['Description']}"))

        # Evaluate and improve prototype design
        prototype_evaluation_prompt = f"Evaluate and suggest improvements for the following prototype design: Design Description: {prototype_design['Design_Description']}, Features: {prototype_design['Features']}, Materials: {prototype_design['Materials']}, Functionality: {prototype_design['Functionality']}, User Interactions: {prototype_design['User_Interactions']}, Testing Plan: {prototype_design['Testing_Plan']}"
        evaluation_and_improvement_prototype = ai(prototype_evaluation_prompt, output_schema=EvaluateAndImprove)
        prototype_design['Design_Description'] = evaluation_and_improvement_prototype['Improved_Design_Description']
        prototype_design['Features'] = evaluation_and_improvement_prototype['Improved_Features']
        prototype_design['Materials'] = evaluation_and_improvement_prototype['Improved_Materials']
        prototype_design['Functionality'] = evaluation_and_improvement_prototype['Improved_Functionality']
        prototype_design['User_Interactions'] = evaluation_and_improvement_prototype['Improved_User_Interactions']
        prototype_design['Testing_Plan'] = evaluation_and_improvement_prototype['Improved_Testing_Plan']

        display(Markdown("#### Updated Prototype Design"))
        display(Markdown(f"**Design Description:** {prototype_design['Design_Description']}"))
        display(Markdown(f"**Features:** {prototype_design['Features']}"))
        display(Markdown(f"**Materials:** {prototype_design['Materials']}"))
        display(Markdown(f"**Functionality:** {prototype_design['Functionality']}"))
        display(Markdown(f"**User Interactions:** {prototype_design['User_Interactions']}"))
        display(Markdown(f"**Testing Plan:** {prototype_design['Testing_Plan']}"))


    # Build a new DataFrame to store data for the successful idea fleshing out
    successful_idea_dataframe = pd.DataFrame({
        'Name': [successful_idea['Name']],
        'Description': [successful_idea['Description']],
        'Potential_Usage': [successful_idea['Potential_Usage']],
        'Unique_Features': [successful_idea['Unique_Features']],
        'Materials_Required': [successful_idea['Materials_Required']],
        'Marketing_Tagline': [marketing_content['Tagline']],
        'Marketing_Description': [marketing_content['Description']],
        'Prototype_Design_Description': [prototype_design['Design_Description']],
        'Prototype_Features': [prototype_design['Features']],
        'Prototype_Materials': [prototype_design['Materials']],
        'Prototype_Functionality': [prototype_design['Functionality']],
        'Prototype_User_Interactions': [prototype_design['User_Interactions']],
        'Prototype_Testing_Plan': [prototype_design['Testing_Plan']]
    })

    return successful_idea_dataframe

openai.api_key = "OpenAI API Key"
api_key_image = "ClipDrop API Key from Stability.ai"

# Set the custom success thresholds
custom_success_thresholds = {
    'feasibility': 5,         #Score needs to be MORE than this number to pass
    'innovation': 5,          #Score needs to be MORE than this number to pass
    'complexity': 9,          #Score needs to be LESS than this number to pass
    'cost': 9,                #Score needs to be LESS than this number to pass
    'market_potential': 5     #Score needs to be MORE than this number to pass
}

# Call the function with desired parameters and custom thresholds
successful_idea = find_unique_invention(
    model_name="gpt-4",
    api_key=openai.api_key,
    invention_category="A product that solves the problem of hair getting caught in the shower drain.",
    max_retries=3,
    success_thresholds=custom_success_thresholds,
    temperature=0.9
)


# Call the function to proceed with the successful idea and create a new DataFrame
successful_idea_df = proceed_with_successful_idea(
    successful_idea,  # Pass the successful idea dataframe here
    model_name="gpt-3.5-turbo-16k",
    api_key= openai.api_key
)

# Print or analyze the new DataFrame for the successful idea
display(Markdown("## DataFrame for Successful Idea Fleshing Out"))
display(successful_idea_df)
successful_idea_df.to_csv("Finalized_Invention.csv")

# Assuming successful_idea_df is a DataFrame containing the final iteration of the invention
final_invention_details = successful_idea_df.iloc[0]
image_prompt = generate_invention_image_prompt(final_invention_details)

filename = final_invention_details['Name']

image = generate_image_from_text(image_prompt, api_key_image, filename)





0


---

## Evaluating invention - Hair Hydro-Phobic Coating Spray

### Invention Description - A hydro-phobic coating spray that, when applied to the hair, makes them resistant to clinging onto wet surfaces. The sprayed hair would still fall off, but instead of getting stuck, they would float on the water making it easier to collect and dispose.

### Scores - Feasibility: 7, Innovation: 8, Complexity: 6, Cost: 6, Market Potential: 8

...

**Decision: Moving on with invention Sonic Cleanse Drain System.**
Reason: Satisfactory criteria met for this invention

---

## Evaluating invention - Bio-Dissolve Hair Cream

### Invention Description - A special cream that can be applied on the body prior to showering. This cream has the ability to dissolve hairs that fall off during the shower, preventing them from reaching the drain in the first place.

KeyboardInterrupt: ignored


# Individually Check Variable Values (for troubleshooting - not part of main script)




In [ ]:
image_prompt

In [ ]:
final_invention_details

Name                                               Sensory Stimulation Puzzle Box
Description                     A puzzle box designed with several tactile, vi...
Potential_Usage                 Enhances cognitive abilities, fine motor skill...
Unique_Features                 Comes with an adjustable difficulty level and ...
Materials_Required              Plastic, electronic components, fabric with di...
Marketing_Tagline               Ignite, Engage, Transform: Discover Your Mind'...
Marketing_Description           The Sensory Stimulation Puzzle Box isn't just ...
Prototype_Design_Description    The Sensory Stimulation Puzzle Box is a multis...
Prototype_Features              1. Multisensory Exploration: The box serves as...
Prototype_Materials             1. Durable, Non-Toxic Plastic: The box's main ...
Prototype_Functionality         The Sensory Stimulation Puzzle Box is a compre...
Prototype_User_Interactions     The Sensory Stimulation Puzzle Box provides a ...
Prototype_Testin

In [ ]:
successful_idea

In [ ]:
successful_idea_df

,Name,Description,Potential_Usage,Unique_Features,Materials_Required,Marketing_Tagline,Marketing_Description,Prototype_Design_Description,Prototype_Features,Prototype_Materials,Prototype_Functionality,Prototype_User_Interactions,Prototype_Testing_Plan
0,Sensory Adventure Mat,A large interactive mat that combines sensory ...,"The sensory adventure mat can be used at home,...",The mat is designed to be large enough for tee...,"Soft fabric, LED lights, speakers, vibrating p...",Sensory Adventure Mat: Unleash the Power of Pe...,The Sensory Adventure Mat takes the ordinary a...,The Sensory Adventure Mat is a meticulously de...,1. Personalized Sensory Experience: The mat of...,1. Premium Quality Fabric: The mat is crafted ...,The Sensory Adventure Mat is more than a tool;...,Users interact with the Sensory Adventure Mat ...,The Sensory Adventure Mat will undergo compreh...


In [ ]:
failed_inventions_prompt

In [ ]:
failed_inventions_df